 # Generación de texto creativo con IA - Poesía - Transformer Decoder Only

### Librerias

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from collections import Counter
import random
import math
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import DatasetDict, concatenate_datasets
import gradio as gr


c:\Users\Victor\anaconda3\envs\pt3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Adquisición de conjunto de datos de poesía, limpieza y pre-procesado

In [2]:
# Cargar el conjunto de datos
dataset = load_dataset("linhd-postdata/poesias")

Se filtra el dataset para usar solo las poesías en español, combinando grupo de train y test para unificar todas las poesías y así poder construir el vocabulario final. A partir de esto se prepara el texto para construir el modelo de lenguaje.

In [3]:
dataset_es = dataset.filter(lambda example: example['language'] == 'es')

In [4]:
merged_dataset = DatasetDict({
    'merged': concatenate_datasets([dataset_es['train'], dataset_es['test']])
})

In [5]:
all_texts = merged_dataset['merged']['text']

A continuación se preparan los datos. Comienza con una lista vacía llamada `words`. Luego, itera sobre una lista de textos llamada `all_texts`, divide cada texto en palabras individuales y las agrega a la lista `words`. Después, cuenta la frecuencia de cada palabra en `words` y crea un vocabulario de todas las palabras únicas. Se asigna a cada palabra un índice único y se crean diccionarios para mapear entre palabras e índices, y viceversa. Se define una longitud de secuencia y se generan muestras de entrenamiento dividiendo el texto en secuencias de palabras de longitud fija, donde cada muestra consiste en una secuencia de palabras seguida por la siguiente palabra en el texto. Esto se realiza para el total de datos y posteriormente para cada particion `train` y `test`.

In [6]:
words = []
for text in all_texts:
    words.extend(text.split())
word_counts = Counter(words)

vocab = list(word_counts.keys())
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}

SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(len(words)-SEQUENCE_LENGTH)]

In [11]:
train_texts = dataset_es['train']['text']
test_texts = dataset_es['test']['text']

In [12]:
words_train = []
for text in train_texts:
    words_train.extend(text.split())
word_counts_train = Counter(words_train)

SEQUENCE_LENGTH = 64
samples_train = [words_train[i:i+SEQUENCE_LENGTH+1] for i in range(len(words_train)-SEQUENCE_LENGTH)]

In [13]:
words_test = []
for text in test_texts:
    words_test.extend(text.split())
word_counts_test = Counter(words_test)

SEQUENCE_LENGTH = 64
samples_test = [words_test[i:i+SEQUENCE_LENGTH+1] for i in range(len(words_test)-SEQUENCE_LENGTH)]

## Funciones y clases para entrenamiento

Se define una clase TextDataset que crea un conjunto de datos personalizado para entrenar el modelo de lenguaje. Toma muestras de texto y un diccionario que mapea palabras a enteros. Devuelve pares de secuencias de palabras, una como entrada y la otra como objetivo, ambas representadas como tensores de PyTorch. Se usa con cada partición  de `train` y `test`.

In [14]:
class TextDataset(Dataset):
    def __init__(self, samples, word_to_int):
        self.samples = samples
        self.word_to_int = word_to_int

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_seq = torch.LongTensor([self.word_to_int[word] for word in sample[:-1]])
        target_seq = torch.LongTensor([self.word_to_int[word] for word in sample[1:]])
        return input_seq, target_seq

In [15]:
BATCH_SIZE = 32
train_dataset = TextDataset(samples_train, word_to_int)
test_dataset = TextDataset(samples_test, word_to_int)

Se usan `data loaders`, los cuales son utilizados en el entrenamiento y evaluación del modelo de lenguaje, permitiendo iterar sobre los datos en lotes de manera eficiente.

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

A continuación se define una función llamada `generate_square_subsequent_mask` que genera una máscara cuadrada para una secuencia. La máscara se utiliza para ocultar posiciones futuras en la secuencia durante el entrenamiento de modelos de lenguaje como Transformers.

In [23]:
def generate_square_subsequent_mask(sz):
    """
    Generate a square mask for the sequence. The masked positions are filled with float('-inf').
    Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

Se define una clase llamada PositionalEncoding que implementa el módulo de codificación posicional en un modelo de Transformer.  Esta clase agrega información sobre la posición absoluta de los tokens a los embeddings de entrada en el modelo, lo que lo ayuda a entender la secuencia y su orden.

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model, dropout=0.1):
        """
        :param max_len: Input length sequence.
        :param d_model: Embedding dimension.
        :param dropout: Dropout value (default=0.1)
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Inputs of forward function
        :param x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """

        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

A continuación la clase TextGen define un modelo de generación de texto basado en Transformer, la cual puede generar secuencias de palabras a partir de una entrada dada. Acá se van cambiando valores de dropout para manejo de regularización.

In [9]:
class TextGen(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads):
        super(TextGen, self).__init__()
        self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim)
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            batch_first=True,
            dropout=0.5
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, vocab_size)
        self.dropout = nn.Dropout(0.5)
        
    # Positional encoding is required. Else the model does not learn.
    def forward(self, x):
        emb = self.emb(x)
        
        # Generate input sequence mask with shape (SEQUENCE_LENGTH, SEQUENCE_LENGTH)
        input_mask = generate_square_subsequent_mask(x.size(1)).to(x.device)
        
        x = self.pos_encoder(emb)
        x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        x = self.dropout(x)
        out = self.linear(x)
        return out

## Entrenamiento y evaluación

Aquí se manejan diferentes hiperparámetros de cara al entrenamiento del modelo, donde despues de diferentes ejercicios se establecieron los que aparecene a continuación.
De cara a manejar la complejidad del modelo se probaron diferentes valores, sobre todo mas bajos en embed_dim, num_layers y num_heads, pero con valores mas bajos los resultados eran inferiores a los obtenidos. Se tienen en total poco mas de 12 millones de parámetros a entrenar.


In [21]:
epochs = 5
learning_rate = 0.001

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextGen(
    vocab_size=vocab_size, 
    embed_dim=100,
    num_layers=2,  
    num_heads=2,
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.0001)

c:\Users\Victor\anaconda3\envs\pt3\lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [23]:
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

TextGen(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (emb): Embedding(298041, 20)
  (decoder_layer): TransformerDecoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=20, out_features=20, bias=True)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=20, out_features=20, bias=True)
    )
    (linear1): Linear(in_features=20, out_features=2048, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=2048, out_features=20, bias=True)
    (norm1): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (dropout3): Dropout(p=0.5, inplace=False)
  )
  (decoder): TransformerDecode

In [36]:
def train(model, epochs, train_dataloader, test_dataloader, criterion, optimizer):
    model.train()
    steps_per_epoch = len(train_dataloader)
    total_steps = epochs * steps_per_epoch
    
    for epoch in range(epochs):
        running_train_loss = 0
        running_test_loss = 0
        
        # Training phase
        for input_seq, target_seq in train_dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            target_seq = target_seq.contiguous().view(-1)
            outputs = outputs.view(-1, vocab_size)
            loss = criterion(outputs, target_seq)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_train_loss += loss.detach().cpu().numpy()
        
        # Calculate and print training loss and perplexity
        epoch_train_loss = running_train_loss / steps_per_epoch
        train_perplexity = math.exp(epoch_train_loss)
        print(f"Epoch {epoch + 1}, train loss: {epoch_train_loss:.3f}, train perplexity: {train_perplexity:.3f}")
        
        # Testing phase
        test_loss, test_perplexity = test(model, test_dataloader, criterion)
        print(f"Epoch {epoch + 1}, test loss: {test_loss:.3f}, test perplexity: {test_perplexity:.3f}")

def test(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            target_seq = target_seq.contiguous().view(-1)
            outputs = outputs.view(-1, vocab_size)
            loss = criterion(outputs, target_seq)
            total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    
    # Calculate perplexity
    perplexity = math.exp(avg_loss)
    
    return avg_loss, perplexity


In [37]:
train(model, epochs, train_dataloader, test_dataloader, criterion, optimizer)

Epoch 1, train loss: 7.585, train perplexity: 1968.358
Epoch 1, test loss: 7.575, test perplexity: 1949.663
Epoch 2, train loss: 5.559, train perplexity: 259.454
Epoch 2, test loss: 7.567, test perplexity: 1934.109
Epoch 3, train loss: 5.379, train perplexity: 216.743
Epoch 3, test loss: 7.600, test perplexity: 1998.170
Epoch 4, train loss: 5.344, train perplexity: 209.390
Epoch 4, test loss: 7.597, test perplexity: 1992.839
Epoch 5, train loss: 5.323, train perplexity: 204.930
Epoch 5, test loss: 7.558, test perplexity: 1916.722


Se guarda el modelo entrenado en el equipo local.

In [38]:
torch.save(model.state_dict(), "C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/modelos_generadores/from_scratch.pth")

### Carga del modelo entrenado

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
the_model = TextGen(embed_dim=100,
    num_layers=2, 
    num_heads=2,
    vocab_size=vocab_size).to(device)
the_model.load_state_dict(torch.load("C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/modelos_generadores/from_scratch.pth"))

<All keys matched successfully>

In [16]:
the_model.eval()

TextGen(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (emb): Embedding(298041, 100)
  (decoder_layer): TransformerDecoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=2048, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=2048, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (dropout3): Dropout(p=0.5, inplace=False)
  )
  (decoder): Transfo

### Inferencia -  Función generadora de poesía

La siguiente función toma un texto como entrada y devuelve un tensor de enteros que representa la secuencia de palabras en el texto. 

In [17]:
def return_int_vector(text):
    words = text.split()
    input_seq = torch.LongTensor([word_to_int[word] for word in words[-SEQUENCE_LENGTH:]]).unsqueeze(0)
    return input_seq

La función sample_next toma las predicciones de un modelo como entrada y devuelve el índice de la siguiente palabra a generar utilizando un enfoque de muestreo aleatorio, para que genere diferentes salidas.

In [19]:
def sample_next(predictions):
    """
    Random sampling.
    """
    # Random sampling approach.
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu().numpy()
    next_token = random.choices(range(len(probabilities[0])), weights=probabilities[0])[0]
    return int(next_token)

La siguiente función genera texto a partir del modelo de lenguaje.

In [64]:
def generativo_texto(sentence, generate_length=100):
    texto_generado = sentence
    the_model.eval()

    for _ in range(generate_length):
        int_vector = return_int_vector(texto_generado)
        
        if len(int_vector) >= SEQUENCE_LENGTH - 1:
            break
        
        input_tensor = int_vector.to(device)
        
        with torch.no_grad():
            predictions = the_model(input_tensor)
        
        next_token = sample_next(predictions)
        texto_generado += ' ' + int_to_word[next_token]

    return texto_generado



#### Interfaz web para generar poesía con Gradio

Usando la libreria Gradio, a partir de la función anterior, se genera una interfaz web que permite ingresar un texto inicial y así generar poesía a partir de dicho texto. La interfaz permite interactuar con ella en el presente notebook o mediante un link web.

In [67]:
interfaz = gr.Interface(
    fn=generativo_texto,
    inputs="text",
    outputs="text",
    title="Generador de Poesía",
    description="Introduce un texto inicial y genera poesía"
)

Se lanza la interfaz web por medio de launch().

In [68]:
interfaz.launch(share=True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://1233ab91bf8d884a0b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Se cierra la interfaz web por medio de close().

In [ ]:
interfaz.close()

Closing server running on port: 7860


In [47]:
sentence = 'Al ver las estrellas pienso en ti'

In [53]:
 s = sentence

In [54]:
type(s)


str

In [55]:
s.split()

['Al', 'ver', 'las', 'estrellas', 'pienso', 'en', 'ti']

In [62]:
for sentence in sentences:
    poesia = texto_generador(sentence)

Ojos asi como  si fueran las así, centinela. He visto desaparecer si he de estrechar juntos, este aire anciano... Déjame en la primavera que callada noche el nido entre labios y frío. No le hablé de piedad tienden suavemente al seductores, No es dable alguna vez que quiero?... Por eso lloro un olor dado que siempre es la costumbre de sentarnos y que vienen aquellas veces trajinada las bulle desvanecido, montada entonces los homosexuales y mandíbula; Días crecen las frentes de las niñas solamente; y las andaban por un solo aire de un río de donde verdea ámame por sus vidas con el




In [63]:
poesia

In [34]:
'Al ver las estrellas pienso en ti'

['Al', 'ver', 'las', 'estrellas', 'pienso', 'en', 'ti']

In [ ]:
def gen_texto(sentence):
    for sentence in sentences:
        print(f"PROMPT: {sentence}")
    return text_generator(sentence, generate_length)

In [ ]:
def genera_texto(sentence):
    for sentence in sentences:
    texto_generador(sentence)


In [36]:
sentences = [
    "Ojos asi como "
]

In [37]:
generate_length = 100

In [38]:
for sentence in sentences:
    print(f"PROMPT: {sentence}")
    text_generator(sentence, generate_length)

PROMPT: Ojos asi como 
Ojos asi como  por los últimos pisos, que ya va propicio que no tenga aureola ni oirás calva sangre con tus virotes la oscura en las que me metas en el fondo del desordenado el hueco de tu mirada, esta noche sin embargo, ya no es tanto como si piensa en el que hay sin tregua con tu puedo y con mi nombre y sobre mi vida perfecta! conocerte, así vengo hasta que raíz alcanzará el alma quisiera en ti. Feliz cumpleaños con el inmediato, milagro de tus crías, encima, en tus cerraduras a la sangre irán las furias vivos a éste, explorando




In [26]:
def gen_texto(sentence):
    for sentence in sentences:
        print(f"PROMPT: {sentence}")
    return text_generator(sentence, generate_length)



In [32]:
gen

PROMPT: Ojos
Ojos que su Jesucristo; Te sabía, mayor, antes que antes cebra divinas despedaza. Dejad que el comunicando tommy derk «Para mí» doble aparato de locura. ¿Querrás lo perdido Que os remeden. y tumbas anónima a la sombra servís del soneto Miran te expulsaron no largos nosotros no está... el alma casa, nos habrá tiempo será formal, de vuestro amado Nos valores y abogados en siglo en tierras lejanas, y sobre la oscura patria amarga condición, Él toda blanca, sin dudar mujer ni libro ni nombre. ¿Cómo expresarme ni don Dinero. Ya le rece su gemido? homicidas a robarse para tú el




PROMPT: Ojos
Ojos por los últimos pisos, porque el tuétano del bosque penetrará al jardín de un instante en una fragua del buey nos Tracia. Me guardan una mano una cinta Labrada con liebre cobarde y divina Otros, que hinche la azucena esbelta, Elsa motivos silba ahogado por las ilusiones, la cima irrupciones de carne sentada en altura de nadar, por la Muerte, hielos, el paisaje con la muerte. rruego; de rosa y una mujer y una estrella. Hoy hiciste desviste de risa, luego, una penumbra plena libertad de garza a fausto y perderlas los jazmines y en los ojos brillan de Neptuno,


PROMPT: Ojos
Ojos que sus estrellas radiantes miras. Pero tiemblen tan morenas pero quizá no casan impenetrables perderá? los ha de trabajar en pura Lo que la vida es dicha mas no es cosa que el verdugo inflame Que censure la primera Azagra suavemente un vivo en su Tenorio, la idea, para él. Y aun saber, que las alas Del frasco Con sus bocas de angustiado en leda suposición Y prados. Tú sobre todo lo debes contar

In [29]:
import gradio as gr

In [30]:
interfaz = gr.Interface(
    fn=gen_texto,
    inputs="text",
    outputs="text",
    title="Generador de Poesía",
    description="Introduce un texto inicial y genera poesía"
)

In [31]:
interfaz.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://c4eb23da9f23aad73e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


PROMPT: Ojos
Ojos e syn peresa; como desides, don Melón para Papa —murmura vuestra memoria? ¿El despertaba quisimos prestar. Trayectorias semiparalelas, fim repartiendo la vida por una herradura la noche solitario y como una voz incapaz muerto, y envió cuando ahora fue juntado, hijo, debo El cortesano es algo». usted he visto entonces me huesos? Osa, mirad, tener una tarde para tener necesidad de contemplar la vida. Y en el sol. Viene a ese filo, crecer Pálida desnuda en boca traducido y mi salvación dormido. Entre la carita del páramo sombrío... Frío mi verte». Y un depositarios Verde sombrío, sin aliento, Todos soltaron


PROMPT: Ojos
Ojos cual los ¿Andas ¡cómo en las cuales pobladores del silencio residen áspides sus huecos el oro de mi raza se mueve para romper el índico nopal, espumas de la zona de las aguas. De Enano entre la noche rápida rizos tan Ejemplo de la playa ya confusa, de plata. Y lloro; y días, senos que se agota, sombra. También busco en piedras y de sol y luna: Natu

In [ ]:
interfaz.close()

Closing server running on port: 7860
